###  Config

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

import random
import numpy as np
import collections
import matplotlib.pyplot as plt
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cpu


### Knapsack Environment

In [2]:
class KnapsackEnv:
    def __init__(self, n_items=10, weights=[], values=[], capacity=300):
        self.n = n_items
        self.weights = weights
        self.values = values
        self.capacity = capacity # np.random.randint(1, sum(self.weights))

    def reset(self):
        self.selected = np.zeros(self.n, dtype=int)
        self.rem_w = self.capacity
        total_value = self.values.sum()

        # Ma trận state (n+1 items, 3 features) [selected, value, weight]
        self.state = np.zeros((self.n+1, 3), dtype=float)
        for i in range(self.n):
            self.state[i] = [
                0,
                self.values[i] / total_value,
                self.weights[i] / self.capacity
            ]
        # Hàng cuối: capacity còn lại (ban đầu normalized = 1)
        self.state[self.n] = [0, 1.0, 1.0]

        return self.state.flatten().astype(np.float32)

    def step(self, action):
        i = action
        reward = 0.0
        done = False

        # check valid
        if self.selected[i] == 0 and self.weights[i] <= self.rem_w:
            self.selected[i] = 1
            self.rem_w -= self.weights[i]
            reward = float(self.values[i])

            # update flag selected
            self.state[i, 0] = 1.0
            # update remaining capacity
            self.state[self.n] = [0, 0.0, self.rem_w / self.capacity]
        else:
            reward = 0.0

        # check done
        feasible = False
        for j in range(self.n):
            if self.selected[j] == 0 and self.weights[j] <= self.rem_w:
                feasible = True
                break
        if not feasible:
            done = True

        return self.state.flatten().astype(np.float32), reward, done, {}

### Select action

In [3]:
def select_action(env, state, q_net, epsilon):
    # epsilon-greedy exploration
    if random.random() < epsilon:
        # explore: chọn ngẫu nhiên trong các action hợp lệ
        valid_actions = []
        for i in range(env.n):
            if env.selected[i] == 0 and env.weights[i] <= env.rem_w:
                valid_actions.append(i)
        if valid_actions:
            return random.choice(valid_actions)
        else:
            return None # không có action hợp lệ
    else:
        # exploit: chọn action có Q lớn nhất
        state_t = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(device)
        q_values = q_net(state_t).detach().cpu().numpy().squeeze()

        # mask: True neu item hop le
        mask = np.zeros(env.n, dtype=bool)
        for i in range(env.n):
            if env.selected[i] == 0 and env.weights[i] <= env.rem_w:
                mask[i] = True

        # neu khong con action hop le
        if not mask.any():
            return None
        q_values[~mask] = -1e9

        # chon action co Q lon nhat
        action = int(np.argmax(q_values))
        return action

### Dueling DQN

In [4]:
class DuelingDQN(nn.Module):
    def __init__(self, input_dim, n_actions, hidden_dim=128):
        super().__init__()
        # phần chung (feature extractor)
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)

        # value stream
        self.v_fc = nn.Linear(hidden_dim, hidden_dim)
        self.v_out = nn.Linear(hidden_dim, 1)

        # advantage stream
        self.a_fc = nn.Linear(hidden_dim, hidden_dim)
        self.a_out = nn.Linear(hidden_dim, n_actions)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        # value branch
        v = F.relu(self.v_fc(x))
        v = self.v_out(v)  # shape: (batch, 1)

        # advantage branch
        a = F.relu(self.a_fc(x))
        a = self.a_out(a)  # shape: (batch, n_actions)

        # combine theo công thức dueling
        q = v + a - a.mean(dim=1, keepdim=True)
        return q

### Replay Buffer

In [5]:
class ReplayBuffer:
    def __init__(self, capacity=100000):
        self.buffer = collections.deque(maxlen=capacity)

    def add(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)
        return (
            np.array(states),
            np.array(actions),
            np.array(rewards, dtype=np.float32),
            np.array(next_states),
            np.array(dones, dtype=np.float32),
        )

    def __len__(self):
        return len(self.buffer)

### Train step

In [6]:
def train_step(online_net, target_net, buffer, optimizer, batch_size=64, gamma=0.99):
    if len(buffer) < batch_size:
        return None  # chưa đủ dữ liệu để train

    # Sample minibatch
    states, actions, rewards, next_states, dones = buffer.sample(batch_size)

    # Convert sang tensor
    states = torch.tensor(states, dtype=torch.float32).to(device)
    actions = torch.tensor(actions, dtype=torch.long).to(device)
    rewards = torch.tensor(rewards, dtype=torch.float32).to(device)
    next_states = torch.tensor(next_states, dtype=torch.float32).to(device)
    dones = torch.tensor(dones, dtype=torch.float32).to(device)

    # Q-values cho actions đã chọn
    q_values = online_net(states).gather(1, actions.unsqueeze(1)).squeeze(1)

    # Double DQN target
    with torch.no_grad():
        next_actions = online_net(next_states).argmax(dim=1)  # chọn action bằng online_net
        next_q = target_net(next_states).gather(1, next_actions.unsqueeze(1)).squeeze(1)
        target = rewards + gamma * next_q * (1 - dones)

    # Loss = MSE
    loss = F.mse_loss(q_values, target)

    # Backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss.item()

### Evaluating

In [11]:
def knapsack_dp(weights, values, capacity):
    n = len(weights)
    dp = [0] * (capacity + 1)
    for i in range(n):
        w, v = weights[i], values[i]
        for cap in range(capacity, w-1, -1):
            dp[cap] = max(dp[cap], dp[cap-w] + v)
    return max(dp)

def knapsack_greedy(weights, values, capacity):
    n = len(weights)
    ratio = [v / w for w, v in zip(weights, values)]
    indices = sorted(range(n), key=lambda k: ratio[k], reverse=True)
    total_value = 0
    total_weight = 0
    for i in indices:
        if total_weight + weights[i] <= capacity:
            total_value += values[i]
            total_weight += weights[i]
    return total_value

def evaluate_graph(env, model, n_eval=100):
    dp_results = []
    greedy_results = []
    agent_results = []
    for _ in tqdm(range(n_eval), desc="Evaluating"):
        state = env.reset()
        done = False
        total_reward = 0
        while not done:
            action = select_action(env, state, model, epsilon=0)
            if action is None:
                break
            state, reward, done, _ = env.step(action)
            total_reward += reward
        dp_result = knapsack_dp(env.weights, env.values, env.capacity)
        greedy_result = knapsack_greedy(env.weights, env.values, env.capacity)
        dp_results.append(dp_result)
        greedy_results.append(greedy_result)
        agent_results.append(total_reward)
    return dp_results, greedy_results, agent_results

def evaluate_agent(env, model, n_eval=100):
    accuracies = []
    for _ in range(n_eval):
        state = env.reset()
        done = False
        total_reward = 0
        while not done:
            action = select_action(env, state, model, epsilon=0)
            if action is None:
                break
            state, reward, done, _ = env.step(action)
            total_reward += reward
        optimal = knapsack_dp(env.weights, env.values, env.capacity)
        acc = total_reward / (optimal + 1e-9)
        accuracies.append(acc)
    return np.mean(accuracies), np.std(accuracies)

## Training Model

In [8]:
all_mean_acc = []
all_std_acc = []
all_weights = []
all_values = []
all_capacity = []
all_online_net = []

for sample in tqdm(range(10), desc="Training"):
    n_items = 10
    episodes = 5000        # để test, bài báo dùng vài ngàn
    buffer_capacity = 50000
    batch_size = 64
    gamma = 0.99
    lr = 1e-3
    # target_update_freq = 100
    epsilon_start = 1.0
    epsilon_end = 0.01
    epsilon_decay = 0.9985  # decay factor

    # Sample data
    weights = np.random.randint(1, 100, n_items)
    values = np.random.randint(1, 100, n_items)
    capacity = np.random.randint(1, sum(weights))

    all_weights.append(weights)
    all_values.append(values)
    all_capacity.append(capacity)

    # Env + networks
    env = KnapsackEnv(n_items=n_items, weights=weights, values=values, capacity=capacity)
    input_dim = (n_items+1) * 3
    n_actions = n_items

    online_net = DuelingDQN(input_dim, n_actions, hidden_dim=64).to(device)
    target_net = DuelingDQN(input_dim, n_actions, hidden_dim=64).to(device)
    target_net.load_state_dict(online_net.state_dict())

    optimizer = optim.Adam(online_net.parameters(), lr=lr)
    scheduler = StepLR(optimizer, step_size=250, gamma=0.9)
    buffer = ReplayBuffer(capacity=buffer_capacity)

    epsilon = epsilon_start

    # Training
    all_rewards = []
    for ep in range(episodes):
        state = env.reset()
        done = False
        total_reward = 0

        while not done:
            action = select_action(env, state, online_net, epsilon)
            if action is None:
                break
            next_state, reward, done, _ = env.step(action)
            buffer.add(state, action, reward, next_state, done)
            state = next_state
            total_reward += reward

            # train step
            loss = train_step(online_net, target_net, buffer, optimizer,
                            batch_size=batch_size, gamma=gamma)

        all_rewards.append(total_reward)
        scheduler.step()
        target_net.load_state_dict(online_net.state_dict())

        # Epsilon decay
        epsilon = max(epsilon_end, epsilon * epsilon_decay)
    all_online_net.append(online_net)

for _ in range(10):
    weights = all_weights[_]
    values = all_values[_]
    capacity = all_capacity[_]
    online_net = all_online_net[_]
    env = KnapsackEnv(n_items=n_items, weights=weights, values=values, capacity=capacity)

    mean_acc, std_acc = evaluate_agent(env, online_net, n_eval=1)
    all_mean_acc.append(mean_acc)
    all_std_acc.append(std_acc)
print("Mean accuracy:", np.mean(all_mean_acc))
print("Std accuracy:", np.mean(all_std_acc))
print("\nTraining done.")

Training:   0%|          | 0/10 [00:00<?, ?it/s]/workspaces/IntroductionToAI/env/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:192: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(
Training: 100%|██████████| 10/10 [11:48<00:00, 70.88s/it]

Mean accuracy: 0.9691064670722953
Std accuracy: 0.0

Training done.


In [13]:
accuracies = []
for _ in range(10):
    weights = all_weights[_]
    values = all_values[_]
    capacity = all_capacity[_]
    optimal = knapsack_dp(env.weights, env.values, env.capacity)
    total_reward = knapsack_greedy(weights, values, capacity)
    
    acc = total_reward / (optimal + 1e-9)
    accuracies.append(acc)

print("Mean accuracy:", np.mean(accuracies))
print("Std accuracy:", np.std(accuracies))

Mean accuracy: 0.6916363636351062
Std accuracy: 0.22600109704951182


In [9]:
# import math

# epsilon_start = 1.0
# epsilon_end = 0.01
# epsilon_mult = 0.9993  # decay factor mỗi episode

# t = math.log(epsilon_end / epsilon_start) / math.log(epsilon_mult)
# print(f"Epsilon đạt {epsilon_end} ở episode ≈ {t:.1f}")
